<a href="https://colab.research.google.com/github/yuichi0625/cellar/blob/main/check_country_language_pair_on_transformers/distilbert_uncased_english.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.0MB 11.7MB/s 
     |████████████████████████████████| 3.2MB 47.2MB/s 
     |████████████████████████████████| 890kB 49.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=e9e978a7c26b3cd5750be8e77d193445cb943f7116d85c63b7612984556f9540
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!wget https://raw.githubusercontent.com/yuichi0625/cellar/main/check_country_language_pair_on_transformers/countries.csv

--2021-03-23 16:40:55--  https://raw.githubusercontent.com/yuichi0625/cellar/main/check_country_language_pair_on_transformers/countries.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7068 (6.9K) [text/plain]
Saving to: ‘countries.csv’

countries.csv       100%[===================>]   6.90K  --.-KB/s    in 0s      

2021-03-23 16:40:55 (104 MB/s) - ‘countries.csv’ saved [7068/7068]



In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
pd.set_option('display.max_rows', 500)

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModelForMaskedLM.from_pretrained('distilbert-base-uncased')

sequence = 'I live in {} and I speak {}.'

rows = []
for name in pd.read_csv('countries.csv')['en'].sort_values():
  input = tokenizer.encode(sequence.format(name, tokenizer.mask_token), return_tensors='pt')
  mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

  token_logits = model(input).logits[0]
  mask_token_logits = token_logits[mask_token_index, :][0]
  mask_token_probs = torch.softmax(mask_token_logits, dim=0)

  candidate_indices = torch.topk(mask_token_logits, k=3, dim=0).indices.tolist()
  result = [f'{tokenizer.decode([ci])} ({mask_token_probs[ci] * 100:.0f}%)' for ci in candidate_indices]

  rows.append([name, *result])

In [4]:
pd.DataFrame(rows, columns=['country', 'language(top1)', 'language(top2)', 'language(top3)'])

,country,language(top1),language(top2),language(top3)
0,Afghanistan,urdu (33%),persian (18%),english (13%)
1,Albania,albanian (77%),english (5%),italian (3%)
2,Algeria,arabic (30%),french (29%),english (9%)
3,Andorra,spanish (49%),catalan (20%),english (7%)
4,Angola,portuguese (54%),english (14%),spanish (9%)
5,Antigua and Barbuda,english (61%),spanish (7%),creole (5%)
6,Argentina,spanish (59%),english (11%),italian (6%)
7,Armenia,armenian (51%),russian (17%),english (5%)
8,Australia,english (27%),spanish (6%),french (5%)
9,Austria,german (20%),french (12%),italian (11%)
